## 1. Overview
This notebook:
1. Clones the repository.
2. Installs dependencies.
3. Uploads secrets (`oci_api_key.pem`, `.env`, `config.properties`, API specs).
4. Starts the MCP Flask app (port 5001).
5. Creates a Cloudflare tunnel for public access.
6. Tests /chat and /mcp/servers endpoints.
Pure Gen AI intent routing defaults to model `cohere.command-plus-latest` in `us-ashburn-1`. Adjust env vars if needed.

In [ ]:
# 2. Diagnostics
import sys, platform
print('Python:', sys.version)
print('Platform:', platform.platform())

In [ ]:
# 3. Clone Repository
import os
REPO_URL = 'https://github.com/Ramsiit2010/OCIGenAIBot.git'
TARGET_DIR = 'OCIGenAIBot'
if not os.path.isdir(TARGET_DIR):
    !git clone $REPO_URL
else:
    print('Repository already present.')

In [ ]:
# 4. Install Dependencies
%cd OCIGenAIBot
!pip install --quiet --upgrade pip
!pip install --quiet -r requirements.txt
print('Dependencies installed.')

## 5. Upload Secrets & Config
Upload any of: `.env`, `oci_api_key.pem`, `config.properties`, `api_spec_*.json`.
If not uploading `.env`, a placeholder will be created.

In [ ]:
from google.colab import files
print('Select files to upload (optional).')
uploaded = files.upload()
for name, data in uploaded.items():
    with open(name,'wb') as f: f.write(data)
print('Uploaded:', list(uploaded.keys()))

## 6. Create Placeholder .env If Missing

In [ ]:
import os
if not os.path.isfile('.env'):
    with open('.env','w') as f:
        f.write(
'''OCI_USER=ocid1.user.oc1..xxxx
OCI_TENANCY=ocid1.tenancy.oc1..xxxx
OCI_FINGERPRINT=aa:bb:...:99
OCI_KEY_FILE=oci_api_key.pem
OCI_REGION=us-ashburn-1
GENAI_REGION=us-ashburn-1
GENAI_MODEL=cohere.command-plus-latest
GENAI_INTENT_MODE=force
'''
        )
    print('.env placeholder created.')
else:
    print('.env found; not modified.')

## 7. Install Cloudflared

In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!sudo dpkg -i cloudflared-linux-amd64.deb || sudo apt-get -f install -y
!cloudflared --version

## 8. Launch RCOEGenAIAgents (Background, Port 5001)

In [ ]:
import subprocess, time
rcoe_proc = subprocess.Popen(['python','RCOEGenAIAgents.py','--port','5001'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
time.sleep(6)
print('RCOEGenAIAgents started on port 5001.')

## 9. Start Cloudflare Tunnel (Expose Port 5001)

In [ ]:
import re, time
tunnel_proc = subprocess.Popen(['cloudflared','tunnel','--url','http://localhost:5001','--no-autoupdate'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
public_url = None
for _ in range(70):
    line = tunnel_proc.stdout.readline()
    if not line: time.sleep(1)
    else:
        print(line.strip())
        m = re.search(r'https://[-a-z0-9]+trycloudflarecom', line)
        if m:
            public_url = m.group(0)
            break
print('
Public Tunnel URL:', public_url or 'NOT FOUND')

## 10. Test /mcp/servers Endpoint

In [ ]:
import requests
assert public_url, 'Tunnel not established.'
servers = requests.get(public_url.rstrip('/') + '/mcp/servers', timeout=30)
print('Status:', servers.status_code)
print('Payload:', servers.json())

## 11. Test /chat Endpoint

In [ ]:
payload = {'prompt':'Show me finance reports'}
chat = requests.post(public_url.rstrip('/') + '/chat', json=payload, timeout=45)
print('Status:', chat.status_code)
print('Response:', chat.json())

## 12. Show Recent Logs (First 50 Lines)

In [ ]:
for i in range(50):
    line = rcoe_proc.stdout.readline()
    if not line: break
    print(line.decode().rstrip())

## 13. Cleanup (Terminate Processes)

In [ ]:
try:
    rcoe_proc.terminate()
    tunnel_proc.terminate()
    print('Processes terminated.')
except Exception as e:
    print('Cleanup error:', e)